In [1]:
import numpy as np
import os
import pandas as pd
import h5py
import seaborn as sns
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from toolbox.utils import set_random_seed
from toolbox.plt_utils import density_scatter

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

In [3]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# for TF To find gpu
os.environ["LD_LIBRARY_PATH"] = f"{os.environ['CONDA_PREFIX']}/lib:" + os.environ.get("LD_LIBRARY_PATH", "")

In [4]:
# set random seed for reproducibility
set_random_seed(seed=42)

2026-01-21 12:34:18.784646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/valehvpa/miniconda3/envs/hist6/lib:/usr/local/cuda/lib64:/usr/local/cuda/lib64
2026-01-21 12:34:18.784672: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow not installed, skipping tf seeding.


/home/valehvpa/miniconda3/envs/hist6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42


## Restore reverse_maya_gapped_peak_histone_closest_2114bp_atac ...

This got lost/overwritten after I moved some some stuff around 😡. So putting it back. Code taken from 12_21_2025.ipynb, 1_17_2026.ipynb, 1_18_2026.ipynb...

In [5]:
base_dir = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_borzoi/histone/peak_scramble"

# read histone peaks (3-column BED assumed: chrom, start, end)
histone_bed = os.path.join(base_dir, "maya_gapped_peaks/gm12878_h3k27ac_peaks.gappedPeak.canonical.sorted.lexicographic")
histone_df = pl.read_csv(
   histone_bed,
    separator="\t",
    has_header=False,
    new_columns=["chrom", "chromStart", "chromEnd"],
).select(["chrom", "chromStart", "chromEnd"])
assert len(histone_df.unique()) == len(histone_df)
print(histone_df.unique().shape, histone_df.shape)

# read chromosome sizes
chr_sizes = "/large_storage/goodarzilab/valehvpa/refs/hg38/hg38.chrom.sizes"
chr_sz_df = pl.read_csv(chr_sizes, separator="\t", has_header=False, new_columns=["chrom", "chrom_size"])

# add chromosome sizes to histone_df
histone_df = histone_df.join(
    chr_sz_df, left_on="chrom", right_on="chrom", how="left",
)
assert histone_df["chrom_size"].is_null().sum() == 0

# add length column
histone_df = histone_df.with_columns((pl.col("chromEnd") - pl.col("chromStart")).alias("length"))

histone_df = histone_df.to_pandas()

(45097, 3) (45097, 3)


In [6]:
def run_closest_for_window_reverse(window: int, base_dir_override: str, atac_bed = None):
    print("*** Running for window size: ", window)

    # read atac peaks, keep only chrom, chromStart, chromEnd
    atac_df = pl.read_csv(
        atac_bed,
        separator="\t",
        has_header=False,
        new_columns=[
            "chrom", "chromStart", "chromEnd", "name", "score", "strand",
        ],
    ).select(["chrom", "chromStart", "chromEnd"])

    # select cols
    pl.from_pandas(histone_df).select(["chrom", "chromStart", "chromEnd"]).write_csv(
        os.path.join(base_dir_override,"temp_B.bed"),
        separator="\t",
        include_header=False
    )
    atac_df.select(["chrom", "chromStart", "chromEnd"]).write_csv(
        os.path.join(base_dir_override,"temp_A.bed"),
        separator="\t",
        include_header=False
    )

    # https://bedtools.readthedocs.io/en/latest/content/tools/closest.html
    fn_prefix = f"reverse_maya_gapped_peak_histone_closest_{str(window)}bp"
    # -t first returns the "first" closest feature in B in case of ties
    cmd = f"cd {base_dir_override} && \
        bedtools closest -a temp_A.bed -b temp_B.bed -t first > {fn_prefix}_atac.bed && \
        command rm -rf temp_A.bed && \
        command rm -rf temp_B.bed"
    os.system(cmd)

In [7]:
bdo = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed"
run_closest_for_window_reverse(
    2114,
    base_dir_override=bdo,
    atac_bed=os.path.join(bdo, "peaks.all_input_regions.ENCSR637XSC.summits.2114bp.sorted.lexicographic.bed.gz"),
)

*** Running for window size:  2114


In [8]:
!head /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_maya_gapped_peak_histone_closest_2114bp_atac.bed

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
chr1	8995	11109	chr1	777901	780141
chr1	777346	779460	chr1	777901	780141
chr1	777610	779724	chr1	777901	780141
chr1	777755	779869	chr1	777901	780141
chr1	778062	780176	chr1	777901	780141
chr1	803509	805623	chr1	804218	805669
chr1	803857	805971	chr1	804218	805669
chr1	804457	806571	chr1	804218	805669
chr1	825985	828099	chr1	826510	828466
chr1	826264	828378	chr1	826510	828466


The two counts below should be teh same (ie number of atac peaks is unchanged after bedtools closest)

In [9]:
!zcat /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/peaks.all_input_regions.ENCSR637XSC.summits.2114bp.sorted.lexicographic.bed.gz | wc -l

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
277907


In [10]:
!wc -l /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_maya_gapped_peak_histone_closest_2114bp_atac.bed

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
277907 /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_maya_gapped_peak_histone_closest_2114bp_atac.bed


In [11]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_maya_gapped_peak_histone_closest_2114bp_atac.bed"
df = pd.read_csv(p, sep="\t", header=None, names=["chrom_atac", "start_atac", "end_atac", "chrom_histone", "start_histone", "end_histone"])
df

,chrom_atac,start_atac,end_atac,chrom_histone,start_histone,end_histone
0,chr1,8995,11109,chr1,777901,780141
1,chr1,777346,779460,chr1,777901,780141
2,chr1,777610,779724,chr1,777901,780141
3,chr1,777755,779869,chr1,777901,780141
4,chr1,778062,780176,chr1,777901,780141
...,...,...,...,...,...,...
277902,chrX,156012422,156014536,chrX,155997182,155998268
277903,chrX,156012694,156014808,chrX,155997182,155998268
277904,chrX,156015787,156017901,chrX,155997182,155998268
277905,chrY,4990899,4993013,.,-1,-1


In [12]:
df[df["chrom_histone"] == "."]

,chrom_atac,start_atac,end_atac,chrom_histone,start_histone,end_histone
277905,chrY,4990899,4993013,.,-1,-1
277906,chrY,15354512,15356626,.,-1,-1


## constrain to 1k distance

skip the atac peaks who are not within Xbp of the hgp (in the sense coded below). Adn for those I set hist_chrom to -1 so that in the code after setting skip_missing_hist these will be effectively skipped.

In [13]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_maya_gapped_peak_histone_closest_2114bp_atac.bed"
atac_hgp_df = pd.read_csv(p, sep="\t", header=None, names=[
    "chrom", "start", "end", "hist_chrom", "hist_start", "hist_end"
])

In [14]:
# if end is < hist_start - 2000, or start > hist_end + 2000, then set out_of_bounds to True
X = 2000
atac_hgp_df["out_of_bounds"] = (atac_hgp_df["end"] < atac_hgp_df["hist_start"] - X) | (atac_hgp_df["start"] > atac_hgp_df["hist_end"] + X)
# if hist_chrom is ".", then also set out_of_bounds to True
atac_hgp_df.loc[atac_hgp_df["hist_chrom"] == ".", "out_of_bounds"] = True
atac_hgp_df["out_of_bounds"].sum() * 100 / len(atac_hgp_df)

45.3288330268759

In [15]:
# if end is < hist_start - 1000, or start > hist_end + 1000, then set out_of_bounds to True
X = 1000
atac_hgp_df["out_of_bounds"] = (atac_hgp_df["end"] < atac_hgp_df["hist_start"] - X) | (atac_hgp_df["start"] > atac_hgp_df["hist_end"] + X)
# if hist_chrom is ".", then also set out_of_bounds to True
atac_hgp_df.loc[atac_hgp_df["hist_chrom"] == ".", "out_of_bounds"] = True
atac_hgp_df["out_of_bounds"].sum() * 100 / len(atac_hgp_df)

46.71023040081754

In [16]:
idx = atac_hgp_df["out_of_bounds"] == True
atac_hgp_df.loc[idx, "hist_chrom"] = "."
atac_hgp_df.loc[idx, "hist_start"] = "-1"
atac_hgp_df.loc[idx, "hist_end"] = "-1"
# set dtype to int
atac_hgp_df["hist_start"] = atac_hgp_df["hist_start"].astype(int)
atac_hgp_df["hist_end"] = atac_hgp_df["hist_end"].astype(int)
del atac_hgp_df["out_of_bounds"]

/tmp/ipykernel_2582215/316362100.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  atac_hgp_df.loc[idx, "hist_start"] = "-1"
/tmp/ipykernel_2582215/316362100.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  atac_hgp_df.loc[idx, "hist_end"] = "-1"


In [17]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_maya_gapped_peak_histone_closest_2114bp_atac_within_1k_bound.bed"
atac_hgp_df.to_csv(
    p,
    sep="\t",
    header=False,
    index=False
)